In [2]:
!pip install -q opencv-python
!pip install -q ultralytics


import torch
import cv2
import numpy as np
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
from ultralytics import YOLO  # YOLOv8 model

# Load pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")  # Downloads automatically if not available

# Known object dimensions (cm) for reference scaling
REFERENCE_OBJECTS = {
    "fork": {"length_cm": 17},  # Average fork length
    "spoon": {"length_cm": 15},
    "plate": {"diameter_cm": 25}
}

# Food density (g/cm³) - Approximate values
FOOD_DENSITY = {
    "apple": 0.8,
    "banana": 0.94,
    "pizza": 0.4,
    "burger": 0.6,
    "steak": 1.1,
    "rice": 0.9,
    "donut": 0.65,
    "sandwich": 0.7
}

# Calories per 100 grams
CALORIE_ESTIMATES = {
    "apple": 52,
    "banana": 89,
    "pizza": 266,
    "burger": 354,
    "steak": 271,
    "rice": 130,
    "donut": 452,
    "sandwich": 250
}

# Load image from URL or file
def load_image(image_path: str):
    try:
        if image_path.startswith('http'):
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(image_path, headers=headers, timeout=10)
            response.raise_for_status()

            image = Image.open(BytesIO(response.content))
            print("✅ Image loaded successfully from URL")
            return image.convert("RGB")

        else:
            image = Image.open(image_path).convert('RGB')
            print("✅ Image loaded successfully from file")
            return image

    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching image: {e}")
    except (UnidentifiedImageError, IOError) as e:
        print(f"❌ Invalid image format: {e}")

    return None

# Estimate weight of food using a known reference object
def estimate_food_weight(image_path):
    image = load_image(image_path)
    if image is None:
        return "Error: Unable to load the image."

    image_cv = np.array(image)
    if image_cv is None or image_cv.size == 0:
        return "Error: Image conversion failed."

    image_cv = cv2.cvtColor(image_cv, cv2.COLOR_RGB2BGR)

    # Perform object detection
    results = model(image_cv)

    # Extract detected objects
    detected_objects = []
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])
            name = model.names[class_id].lower()
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            width = x2 - x1
            height = y2 - y1
            detected_objects.append({"name": name, "width": width, "height": height})

    print(f"🔍 Detected objects: {detected_objects}")

    if not detected_objects:
        return "Error: No objects detected in the image."

    # Find food and reference object
    food_items = [obj for obj in detected_objects if obj["name"] in FOOD_DENSITY]
    reference_objects = [obj for obj in detected_objects if obj["name"] in REFERENCE_OBJECTS]

    if not food_items:
        return "No recognizable food item detected."

    # If no reference object, try using the dining table
    if not reference_objects:
        print("⚠️ No fork, spoon, or plate detected. Trying to use dining table as reference...")
        reference_object = next((obj for obj in detected_objects if obj["name"] == "dining table"), None)

        if reference_object:
            REFERENCE_OBJECTS["dining table"] = {"length_cm": 75}
            reference_objects.append(reference_object)  # Use the dining table as a reference object
        else:
            print("⚠️ No valid reference object detected. Try including a fork, spoon, or plate in the image.")
            return "No reference object detected (fork, spoon, plate, or table required)."

    # Pick the largest food item
    food_item = max(food_items, key=lambda x: x["width"] * x["height"])

    # Pick the best reference object
    reference_object = max(reference_objects, key=lambda x: x["width"] * x["height"])

    # Calculate pixel-to-cm ratio
    ref_cm = REFERENCE_OBJECTS[reference_object["name"]]["length_cm"]
    ref_px = reference_object["height"]
    pixel_per_cm = ref_px / ref_cm

    # Adjust depth estimation based on food type
    food_width_cm = food_item["width"] / pixel_per_cm
    food_height_cm = food_item["height"] / pixel_per_cm
    if food_item["name"] in ["pizza", "steak"]:
        food_depth_cm = 1
    elif food_item["name"] in ["burger", "donut", "sandwich"]:
        food_depth_cm = food_width_cm * 0.6
    else:
        food_depth_cm = min(food_width_cm, food_height_cm) * 0.5

    volume_cm3 = food_width_cm * food_height_cm * food_depth_cm

    food_name = food_item["name"]
    density = FOOD_DENSITY.get(food_name, 1.0)
    estimated_weight = volume_cm3 * density

    print(f"📏 Estimated weight of {food_name}: {estimated_weight:.2f}g")

    return food_name, estimated_weight

# Estimate calories based on food weight
def estimate_calories(image_path):
    result = estimate_food_weight(image_path)

    if isinstance(result, str):
        return result

    food_name, estimated_weight = result

    calorie_per_100g = CALORIE_ESTIMATES.get(food_name, None)

    if calorie_per_100g is None:
        return f"Detected {food_name}. Estimated weight: {estimated_weight:.2f}g. Calorie data unavailable."

    total_calories = (calorie_per_100g / 100) * estimated_weight

    print(f"🔥 Estimated calories for {food_name}: {total_calories:.2f} kcal")

    return f"🍔 Detected {food_name}. Estimated weight: {estimated_weight:.2f}g. Estimated calories: {total_calories:.2f} kcal."

# Example usage
image_url = "https://upload.wikimedia.org/wikipedia/commons/a/a3/Eq_it-na_pizza-margherita_sep2005_sml.jpg"
print(estimate_calories(image_url))


✅ Image loaded successfully from URL

0: 480x640 1 pizza, 1 dining table, 12.0ms
Speed: 2.9ms preprocess, 12.0ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
🔍 Detected objects: [{'name': 'pizza', 'width': 978, 'height': 676}, {'name': 'dining table', 'width': 1000, 'height': 767}]
⚠️ No fork, spoon, or plate detected. Trying to use dining table as reference...
📏 Estimated weight of pizza: 2528.58g
🔥 Estimated calories for pizza: 6726.03 kcal
🍔 Detected pizza. Estimated weight: 2528.58g. Estimated calories: 6726.03 kcal.
